### Criar um secret scope

1º - Criar o secret no Key Vault da Azure

2º - Acessar o endereço "https://(databricks-instance-here)#secrets/createScope" colocando a sua instacia do databricks

3º - Preencher com as informações soicitadas
  
https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes  

###Montar um blob

In [0]:
dbutils.fs.mount(
  source = "wasbs://dax@ricafunctionapp.blob.core.windows.net",
  mount_point = "/mnt/blob/dax",
  extra_configs = {"fs.azure.account.key.ricafunctionapp.blob.core.windows.net":dbutils.secrets.get(scope = "rica-secret-scope", key = "ricafunctionapp-key")})

  # https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts

True

In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

##Leitura e transformação

In [0]:
df = (spark
        .read
        .json('dbfs:/mnt/blob/dax*'))

# display(df)

In [0]:
df_exp = df.select(
    'data.symbol',
    'data.ask',
    'data.baseVolume24h',
    'data.bid',
    'data.high24h',
    'data.lastPrice',
    'data.low24h',
    'data.open24h',
    'data.quoteVolume24h',
    'data.timestamp'
)
# display(df_exp)

In [0]:
df_exp = (
    df_exp
    .withColumn("timestamp", from_unixtime(df_exp.timestamp / 1000))
    .withColumn("broker_id", lit("1"))
    .withColumn("usuario_id", lit("1"))
    )
# display(df_exp)

In [0]:
df = df_exp.select(
    'broker_id',
    'usuario_id',
    'symbol',
    'ask',
    'baseVolume24h',
    'bid',
    'high24h',
    'lastPrice',
    'low24h',
    'open24h',
    'quoteVolume24h',
    'timestamp'
)
# display(df)

##Save into AzureSQL DB

In [0]:
jdbcUser = 'user'
jdbcPassword  = 'password'
jdbcHostname = 'server.database.windows.net'
jdbcDatabase = 'db'
jdbcPort = 1433
jdbcDriver = 'com.microsoft.sqlserver.jdbc.SQLServerDriver'

jdbcUrl = 'jdbc:sqlserver://{0}:{1};database={2}'.format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  'user' : jdbcUser,
  'password' : jdbcPassword,
  'driver' : jdbcDriver
}

table = 'dax_api'

(df
   .write.mode('append')
  .format('jdbc')
  .option('url', jdbcUrl)
  .option('dbtable', table)
  .option('user', jdbcUser)
  .option('password', jdbcPassword)
  .option('driver', jdbcDriver)
  .save()
)